In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

 

## Download data and parse it

In [7]:

r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

## Get Headers

In [8]:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

## find all the items and skip first one

In [9]:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

## skip all meta symbols and line feeds

In [10]:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

## make dataframe and expand rows

In [11]:

df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

## skip not assigned boroughs

In [12]:
df = df.drop(df[(df.Borough == "Not assigned")].index)

## give "not assigned" same name as boroughs

In [13]:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

## copy borough value to neighbourhood if NaN

In [14]:
df.Neighbourhood.fillna(df.Borough, inplace=True)

## drop duplicate rows

In [15]:
df=df.drop_duplicates()

In [16]:
## delete toronto annotation fom neighbourhood
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))
df.update(
    df.Borough.loc[
       lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

In [17]:
#print(df);
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes
df2.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park (Toronto)
5,M9A,Queen's Park (Toronto),Queen's Park (Toronto)
6,M1B,"Scarborough, Toronto","Malvern, Toronto, Rouge, Toronto"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [19]:
df2.shape

(103, 3)